# Failure to Mix - Complete Experiment Runner

This notebook runs all experiments and generates all figures for the paper:
- Figure 1: Single flip baseline (8 models)
- Figure 2: Multiple decisions D=2, D=3
- Figure 3: D=10 ensemble analysis
- Figure 4: Ternary distribution (0/1/2)
- Figure 5: Word bias experiments
- Figure 6: Game theory applications

In [ ]:
# Install dependencies
!pip -q install aiohttp pandas matplotlib google-api-python-client tqdm nest_asyncio

In [ ]:
import asyncio
import aiohttp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import time
import datetime
import traceback
from tqdm import tqdm
import nest_asyncio
nest_asyncio.apply()

# Google Drive integration (optional)
try:
    from google.colab import auth
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaFileUpload
    auth.authenticate_user()
    drive = build('drive', 'v3')
    sheets = build('sheets', 'v4')
    DRIVE_ENABLED = True
except:
    DRIVE_ENABLED = False
    print("Google Drive not available, saving locally only")

## Configuration

In [ ]:
# API Configuration
API_KEYS = [
    "sk-or-v1-YOUR-KEY-1",  # Replace with your OpenRouter API keys
    "sk-or-v1-YOUR-KEY-2",
]
API_URL = "https://openrouter.ai/api/v1/chat/completions"

# Experiment settings
BATCH_SIZE = 20
INITIAL_WAIT = 2.0
MAX_RETRIES = 3
SAVE_PATH = "/content/results"
os.makedirs(SAVE_PATH, exist_ok=True)

# Model colors (matching paper)
MODEL_COLORS = {
    "google/gemini-2.5-pro": "#1f77b4",       # Blue
    "openai/gpt-5": "#2ca02c",                 # Green
    "openai/gpt-5-nano": "#d62a28",            # Red
    "anthropic/claude-4.5-sonnet": "#8c564b",  # Brown-purple
    "moonshotai/kimi-k2-0905": "#e377c2",      # Pink
    "qwen/qwen3-vl-8b-instruct": "#662d91",    # Purple
    "x-ai/grok-4-fast": "#c2b59b",             # Tan
    "deepseek/deepseek-v3.2": "#9467bd",       # Purple
}

EDGE_COLOR = "#999b9e"

def get_color(model):
    model_lower = model.lower()
    for key, color in MODEL_COLORS.items():
        if key.lower() in model_lower or model_lower in key.lower():
            return color
    # Provider fallback
    if "gemini" in model_lower: return "#1f77b4"
    if "gpt" in model_lower:
        if "nano" in model_lower: return "#d62a28"
        return "#2ca02c"
    if "claude" in model_lower or "sonnet" in model_lower: return "#8c564b"
    if "kimi" in model_lower: return "#e377c2"
    if "qwen" in model_lower: return "#662d91"
    if "grok" in model_lower: return "#c2b59b"
    if "deepseek" in model_lower: return "#9467bd"
    return "#7f7f7f"

## Core Functions

In [ ]:
# Response parsing
def parse_response(d: dict):
    """Parse API response to extract content and reasoning."""
    msg = (d.get("choices") or [{}])[0].get("message", {})
    content = msg.get("content") or ""
    reasoning = msg.get("reasoning") or ""
    if not reasoning:
        rd = msg.get("reasoning_details")
        if isinstance(rd, list) and rd:
            reasoning = rd[0].get("summary") or rd[0].get("data") or ""
    merged = (content or reasoning or "").strip()
    return content.strip(), reasoning.strip(), merged

def extract_binary(text: str) -> str:
    """Extract 0/1 from response."""
    if not text: return "error"
    t = text.strip()
    if t == "0": return "0"
    if t == "1": return "1"
    m = re.findall(r"\b([01])\b", t)
    if m: return m[-1]
    if "1" in t and "0" not in t: return "1"
    if "0" in t and "1" not in t: return "0"
    low = t.lower()
    if "one" in low and "zero" not in low: return "1"
    if "zero" in low and "one" not in low: return "0"
    return "error"

def extract_heads_tails(text: str) -> str:
    """Extract Heads/Tails from response."""
    if not text: return "error"
    m = re.findall(r"(Heads|Tails)", text, re.IGNORECASE)
    if m: return m[-1].capitalize()
    return "error"

def extract_ternary(text: str) -> str:
    """Extract 0/1/2 from response."""
    if not text: return "error"
    t = text.strip()
    if t in ["0", "1", "2"]: return t
    m = re.findall(r"\b([012])\b", t)
    if m: return m[-1]
    return "error"

def extract_word(text: str, word1: str, word2: str) -> str:
    """Extract word choice from response."""
    if not text: return "error"
    t = text.strip().lower()
    w1, w2 = word1.lower(), word2.lower()
    if t == w1: return word1
    if t == w2: return word2
    pattern = f"({re.escape(w1)}|{re.escape(w2)})"
    m = re.findall(pattern, t, re.IGNORECASE)
    if m:
        return word1 if m[-1].lower() == w1 else word2
    return "error"

# S metric calculation
def compute_S(p_vals, r_vals, extend=True):
    """Compute step-likeness metric S."""
    p = np.asarray(p_vals, dtype=float)
    r = np.asarray(r_vals, dtype=float)
    mask = np.isfinite(p) & np.isfinite(r)
    p, r = p[mask], r[mask]
    if len(p) < 2: return np.nan
    order = np.argsort(p)
    p, r = p[order], r[order]
    # Convert to [0,1] if percentage
    if p.max() > 1:
        p = p / 100.0
        r = r / 100.0
    # Extend to boundaries
    if extend:
        if p[0] > 0:
            p = np.insert(p, 0, 0)
            r = np.insert(r, 0, r[0])
        if p[-1] < 1:
            p = np.append(p, 1)
            r = np.append(r, r[-1])
    # Trapezoid integration
    dp = np.diff(p)
    left = np.abs(r[:-1] - p[:-1])
    right = np.abs(r[1:] - p[1:])
    trap = np.sum(dp * (left + right) / 2.0)
    return 4.0 * trap

In [ ]:
# API caller
async def call_model(session, model, messages, api_key, max_tokens=2, temperature=None, attempt=1):
    """Make API call with retry logic."""
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    payload = {"model": model, "messages": messages}
    if max_tokens: payload["max_tokens"] = max_tokens
    if temperature is not None: payload["temperature"] = temperature
    
    try:
        async with session.post(API_URL, json=payload, headers=headers, timeout=120) as r:
            if r.status == 429:
                if attempt < MAX_RETRIES:
                    await asyncio.sleep(3 * attempt)
                    return await call_model(session, model, messages, api_key, max_tokens, temperature, attempt+1)
                return {"status": "rate_limited", "content": ""}
            d = await r.json()
            content, reasoning, merged = parse_response(d)
            return {"status": "success", "content": content, "reasoning": reasoning, "raw": d}
    except Exception as e:
        if attempt < MAX_RETRIES:
            await asyncio.sleep(2 * attempt)
            return await call_model(session, model, messages, api_key, max_tokens, temperature, attempt+1)
        return {"status": str(e), "content": ""}

## Figure 1: Single Flip Baseline

Tests 8 models on simple binary coin flip with p from 0-100%

In [ ]:
# Figure 1 - Single flip experiment
MODELS_FIG1 = [
    "google/gemini-2.5-pro",
    "openai/gpt-5",
    "openai/gpt-5-nano",
    "anthropic/claude-4.5-sonnet",
    "moonshotai/kimi-k2-0905",
    "qwen/qwen3-vl-8b-instruct",
    "x-ai/grok-4-fast",
    "deepseek/deepseek-v3.2",
]

PROMPT_SINGLE_FLIP = 'I\'d like you to flip a biased coin that returns "1" {p}% of the time and "0" {100_p}% of the time. Please return a 1 token response simulating a single flip.'

async def run_fig1_experiment(model, p_values, n_trials=100):
    """Run single flip experiment for one model."""
    results = []
    async with aiohttp.ClientSession() as session:
        for p in tqdm(p_values, desc=f"{model.split('/')[-1]}"):
            prompt = PROMPT_SINGLE_FLIP.format(p=p, **{"100_p": 100-p})
            for trial in range(n_trials):
                api_key = API_KEYS[(p * n_trials + trial) % len(API_KEYS)]
                resp = await call_model(
                    session, model,
                    [{"role": "user", "content": prompt}],
                    api_key
                )
                answer = extract_binary(resp["content"])
                results.append({"p": p, "trial": trial, "answer": answer, "status": resp["status"]})
                await asyncio.sleep(INITIAL_WAIT / BATCH_SIZE)
    return pd.DataFrame(results)

def analyze_fig1(df):
    """Compute r for each p value."""
    valid = df[df["answer"].isin(["0", "1"])]
    summary = valid.groupby("p").apply(lambda x: (x["answer"]=="1").mean() * 100).reset_index()
    summary.columns = ["p", "r"]
    return summary

In [ ]:
# Plot Figure 1c - Full Gemini curve
def plot_fig1c(summary, model_name, save_path=None):
    """Plot full p-r curve (0-100)."""
    fig, ax = plt.subplots(figsize=(8, 6))
    p, r = summary["p"].values, summary["r"].values
    color = get_color(model_name)
    
    ax.plot(p, r, color=color, linewidth=1.8, zorder=2)
    ax.scatter(p, r, s=50, facecolors=color, edgecolors='black', linewidths=0.6, zorder=3)
    ax.plot([0, 100], [0, 100], '--', color='gray', alpha=0.5, linewidth=0.8)
    ax.axvline(x=50, color=EDGE_COLOR, linewidth=0.9, alpha=0.3)
    
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.set_xlabel("p (%)", fontweight='bold')
    ax.set_ylabel("r (%)", fontweight='bold')
    ax.set_title(f"{model_name}", fontweight='bold')
    
    for spine in ax.spines.values():
        spine.set_color(EDGE_COLOR)
    ax.tick_params(color=EDGE_COLOR, labelcolor=EDGE_COLOR)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight', transparent=True)
        plt.savefig(save_path.replace('.png', '.svg'), format='svg', bbox_inches='tight')
    plt.show()
    return fig

# Plot Figure 1e - Zoomed comparison (45-55)
def plot_fig1e(all_results, save_path=None):
    """Plot 8 models zoomed to p=45-55."""
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for idx, (model, summary) in enumerate(all_results.items()):
        ax = axes[idx]
        mask = (summary["p"] >= 45) & (summary["p"] <= 55)
        sub = summary[mask]
        color = get_color(model)
        
        ax.plot(sub["p"], sub["r"], 'o-', color=color, markersize=8,
                markeredgecolor='black', markeredgewidth=0.6)
        ax.axvline(x=50, color=EDGE_COLOR, linewidth=0.9, alpha=0.3)
        
        ax.set_xlim(45, 55)
        ax.set_ylim(0, 100)
        ax.set_title(model.split('/')[-1], fontsize=11, color=color)
        ax.set_xlabel("p (%)")
        ax.set_ylabel("r (%)")
        
        for spine in ax.spines.values():
            spine.set_color(EDGE_COLOR)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    return fig

## Figure 2: Multiple Decisions (D=2, D=3)

In [ ]:
# Figure 2 - Multiple decisions
PROMPT_MULTI_FLIP = 'I\'d like you to flip a biased coin that returns "1" {p}% of the time and "0" {100_p}% of the time. Please return a {D} token response simulating {D} flips. Please return exactly {D} tokens (0 or 1) separated by a single space.'

async def run_fig2_experiment(model, p_values, D=2, n_trials=100):
    """Run multi-flip experiment."""
    results = []
    async with aiohttp.ClientSession() as session:
        for p in tqdm(p_values, desc=f"D={D}"):
            prompt = PROMPT_MULTI_FLIP.format(p=p, D=D, **{"100_p": 100-p})
            for trial in range(n_trials):
                api_key = API_KEYS[(p * n_trials + trial) % len(API_KEYS)]
                resp = await call_model(
                    session, model,
                    [{"role": "user", "content": prompt}],
                    api_key,
                    max_tokens=D * 2 + 5
                )
                # Parse D responses
                text = resp["content"]
                if " " in text:
                    tokens = text.split()
                else:
                    tokens = list(text)
                tokens = [t for t in tokens if t in ["0", "1"]]
                tokens = (tokens + [None] * D)[:D]  # Pad/truncate
                
                for j, tok in enumerate(tokens):
                    results.append({
                        "p": p, "trial": trial, "D": D, "j": j+1,
                        "answer": tok if tok else "error",
                        "status": resp["status"]
                    })
                await asyncio.sleep(INITIAL_WAIT / BATCH_SIZE)
    return pd.DataFrame(results)

def analyze_fig2(df, j=None):
    """Compute r for specific j or mean across all j."""
    if j is not None:
        sub = df[df["j"] == j]
    else:
        sub = df
    valid = sub[sub["answer"].isin(["0", "1"])]
    summary = valid.groupby("p").apply(lambda x: (x["answer"]=="1").mean() * 100).reset_index()
    summary.columns = ["p", "r"]
    return summary

In [ ]:
# Plot Figure 2b - j=1 response (step function)
def plot_fig2b(summary, model_name, D, save_path=None):
    """Plot first decision response curve."""
    fig, ax = plt.subplots(figsize=(8, 6))
    p, r = summary["p"].values, summary["r"].values
    S = compute_S(p, r)
    
    ax.plot(p, r, 'o-', color=get_color(model_name), markersize=6)
    ax.plot([0, 100], [0, 100], '--', color='gray', alpha=0.5)
    
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.set_xlabel("p (%)")
    ax.set_ylabel("r (%)")
    ax.set_title(f"{model_name}\nD = {D}, j = 1 (first output)\nS = {S:.3f}")
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig, S

# Plot Figure 2c - j=2 zigzag
def plot_fig2c(all_j2_results, D=2, save_path=None):
    """Plot second decision zigzag for multiple models."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for model, summary in all_j2_results.items():
        ax.plot(summary["p"], summary["r"], 'o-', label=model.split('/')[-1],
                color=get_color(model), markersize=5)
    
    ax.axvline(x=50, color=EDGE_COLOR, linewidth=0.9, alpha=0.3)
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.set_xlabel("p (%)")
    ax.set_ylabel("r (%)")
    ax.set_title(f"D = {D}, j = 2 (second output)")
    ax.legend()
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig

# Plot Figure 2e - Mean(r) comparison
def plot_fig2e(mean_results_d2, mean_results_d3, save_path=None):
    """Plot mean response curves for D=2 and D=3."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for ax, (results, D) in zip(axes, [(mean_results_d2, 2), (mean_results_d3, 3)]):
        for model, summary in results.items():
            S = compute_S(summary["p"].values, summary["r"].values)
            ax.plot(summary["p"], summary["r"], 'o-',
                    label=f"{model.split('/')[-1]} (S={S:.3f})",
                    color=get_color(model), markersize=4)
        
        ax.plot([0, 100], [0, 100], '--', color='gray', alpha=0.5)
        ax.set_xlim(0, 100)
        ax.set_ylim(0, 100)
        ax.set_xlabel("p (%)")
        ax.set_ylabel("Mean(r) (%)")
        ax.set_title(f"Mean(r), D = {D}")
        ax.legend(fontsize=8)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig

## Figure 3: D=10 Ensemble Analysis

In [ ]:
# Figure 3 - D=10 ensemble
from scipy.stats import binom

async def run_fig3_experiment(model, p_values, D=10, n_trials=1000):
    """Run D=10 experiment with more trials."""
    # Same as fig2 but with D=10 and N=1000
    return await run_fig2_experiment(model, p_values, D=D, n_trials=n_trials)

def plot_fig3a(df, model_name, save_path=None):
    """Plot individual j response curves for D=10."""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    for ax, j in zip(axes.flatten(), [1, 2, 9, 10]):
        summary = analyze_fig2(df, j=j)
        S = compute_S(summary["p"].values, summary["r"].values)
        
        ax.plot(summary["p"], summary["r"], 'o-', color=get_color(model_name))
        ax.plot([0, 100], [0, 100], '--', color='gray', alpha=0.5)
        ax.set_xlim(0, 100)
        ax.set_ylim(0, 100)
        ax.set_title(f"D = 10, j = {j}\nS = {S:.3f}")
        ax.set_xlabel("p (%)")
        ax.set_ylabel("r (%)")
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig

def plot_fig3c(all_model_dfs, p_test, D=10, save_path=None):
    """Plot histogram of sum of 1s for specific p values."""
    n_models = len(all_model_dfs)
    fig, axes = plt.subplots(2, n_models, figsize=(4*n_models, 8))
    
    p_vals = [15, 45]  # Two p values to test
    
    for row, p in enumerate(p_vals):
        for col, (model, df) in enumerate(all_model_dfs.items()):
            ax = axes[row, col]
            
            # Count sum of 1s per trial
            trial_sums = df[df["p"] == p].groupby("trial").apply(
                lambda x: (x["answer"] == "1").sum()
            )
            
            # Expected binomial
            x = np.arange(D + 1)
            expected = binom.pmf(x, D, p/100) * len(trial_sums)
            observed, _ = np.histogram(trial_sums, bins=np.arange(D + 2) - 0.5)
            
            ax.bar(x - 0.2, expected, width=0.4, label='Expected i.i.d.', alpha=0.7)
            ax.bar(x + 0.2, observed, width=0.4, label='Observed', alpha=0.7,
                   color=get_color(model))
            
            ax.set_xlabel("Number of '1' outcomes")
            ax.set_ylabel("Frequency")
            ax.set_title(f"{model.split('/')[-1]}\np = {p/100}")
            if row == 0 and col == 0:
                ax.legend()
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig

## Figure 4: Ternary Distribution

In [ ]:
# Figure 4 - Ternary distribution (0/1/2)
PROMPT_TERNARY = 'I\'d like to draw from a biased deck that returns "2" {q}% of the time, "1" {p}% of the time and "0" {r}% of the time. Please return a 1 token response simulating a single draw.'

async def run_fig4_experiment(model, q_values, p_fixed=40, n_trials=100):
    """Run ternary distribution experiment."""
    results = []
    async with aiohttp.ClientSession() as session:
        for q in tqdm(q_values, desc=f"Ternary"):
            r_val = 100 - p_fixed - q
            if r_val < 0:
                continue
            prompt = PROMPT_TERNARY.format(q=q, p=p_fixed, r=r_val)
            
            for trial in range(n_trials):
                api_key = API_KEYS[(q * n_trials + trial) % len(API_KEYS)]
                resp = await call_model(
                    session, model,
                    [{"role": "user", "content": prompt}],
                    api_key
                )
                answer = extract_ternary(resp["content"])
                results.append({
                    "q": q, "p_fixed": p_fixed, "trial": trial,
                    "answer": answer, "status": resp["status"]
                })
                await asyncio.sleep(INITIAL_WAIT / BATCH_SIZE)
    return pd.DataFrame(results)

def analyze_fig4(df):
    """Compute r0, r1, r2 for each q value."""
    valid = df[df["answer"].isin(["0", "1", "2"])]
    summary = valid.groupby("q").apply(
        lambda x: pd.Series({
            "r0": (x["answer"] == "0").mean() * 100,
            "r1": (x["answer"] == "1").mean() * 100,
            "r2": (x["answer"] == "2").mean() * 100,
        })
    ).reset_index()
    return summary

def plot_fig4(all_results, p_fixed=40, save_path=None):
    """Plot Figure 4b, 4c, 4d - ternary response curves."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Plot r0, r1, r2
    for ax, (response, ylabel) in zip(axes, 
            [("r0", "R = 0 (response value)"), 
             ("r1", "R = 1 (response value)"),
             ("r2", "R = 2 (response value)")]):
        
        # Expected values
        q_range = np.arange(0, 61, 5)
        if response == "r0":
            expected = 100 - p_fixed - q_range
        elif response == "r1":
            expected = np.full_like(q_range, p_fixed, dtype=float)
        else:
            expected = q_range
        
        ax.plot(q_range, expected, '--', color='gray', label='Expected', linewidth=1)
        
        # Green shaded region where this response has highest probability
        if response == "r0":
            ax.axvspan(0, 20, facecolor='lightgreen', alpha=0.3)
        elif response == "r1":
            ax.axvspan(20, 40, facecolor='lightgreen', alpha=0.3)
        else:
            ax.axvspan(40, 60, facecolor='lightgreen', alpha=0.3)
        
        # Plot each model
        for model, summary in all_results.items():
            ax.plot(summary["q"], summary[response], 'o-',
                    label=model.split('/')[-1], color=get_color(model), markersize=5)
        
        ax.set_xlim(0, 60)
        ax.set_ylim(0, 100)
        ax.set_xlabel("q (%)")
        ax.set_ylabel(f"r (%) - {ylabel}")
        ax.set_title(f"p = {p_fixed}%\n{ylabel}")
        ax.legend(fontsize=8)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig

## Figure 5: Word Bias

In [ ]:
# Figure 5 - Word bias
WORD_PAIRS = [
    ("luxury", "affordable"),
    ("1", "0"),
    ("adult", "child"),
    ("sun", "moon"),
    ("black", "white"),
    ("cat", "dog"),
    ("woman", "man"),
    ("rich", "poor"),
    ("human", "alien"),
    ("day", "night"),
    ("37", "50"),
]

PROMPT_WORD = 'I\'d like you to flip a biased coin that returns "{w1}" {p}% of the time and "{w2}" {100_p}% of the time. Please return a 1 token response simulating a single flip.'

async def run_fig5_experiment(model, word_pairs, p=50, n_trials=100):
    """Run word bias experiment."""
    results = []
    async with aiohttp.ClientSession() as session:
        for w1, w2 in tqdm(word_pairs, desc="Word pairs"):
            for reversed_order in [False, True]:
                if reversed_order:
                    prompt = PROMPT_WORD.format(w1=w2, w2=w1, p=100-p, **{"100_p": p})
                else:
                    prompt = PROMPT_WORD.format(w1=w1, w2=w2, p=p, **{"100_p": 100-p})
                
                for trial in range(n_trials):
                    api_key = API_KEYS[trial % len(API_KEYS)]
                    resp = await call_model(
                        session, model,
                        [{"role": "user", "content": prompt}],
                        api_key, max_tokens=10
                    )
                    answer = extract_word(resp["content"], w1, w2)
                    results.append({
                        "word1": w1, "word2": w2,
                        "reversed": reversed_order,
                        "trial": trial,
                        "answer": answer,
                        "chose_word1": answer == w1,
                        "status": resp["status"]
                    })
                    await asyncio.sleep(INITIAL_WAIT / BATCH_SIZE)
    return pd.DataFrame(results)

def analyze_fig5(df):
    """Compute bias for each word pair."""
    results = []
    for (w1, w2), group in df.groupby(["word1", "word2"]):
        normal = group[~group["reversed"]]
        reversed_df = group[group["reversed"]]
        
        r_normal = normal["chose_word1"].mean() * 100
        r_reversed = reversed_df["chose_word1"].mean() * 100
        avg_bias = (r_normal + r_reversed) / 2
        
        results.append({
            "word1": w1, "word2": w2,
            "r_word1_first": r_normal,
            "r_word1_second": r_reversed,
            "avg_bias": avg_bias
        })
    return pd.DataFrame(results)

def plot_fig5bc(summary, model_name, save_path=None):
    """Plot Figure 5b and 5c - word bias bars."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    labels = [f"{r['word1']}/{r['word2']}" for _, r in summary.iterrows()]
    y = np.arange(len(labels))
    
    # 5b - word1 first
    axes[0].barh(y, summary["r_word1_first"], color='salmon')
    axes[0].set_yticks(y)
    axes[0].set_yticklabels(labels)
    axes[0].set_xlim(0, 100)
    axes[0].set_xlabel("Response rate for red word (%)")
    axes[0].set_title(f"{model_name}\nRed word listed first")
    axes[0].axvline(x=50, color='gray', linestyle='--')
    
    # 5c - word1 second (reversed)
    axes[1].barh(y, summary["r_word1_second"], color='lightblue')
    axes[1].set_yticks(y)
    axes[1].set_yticklabels(labels)
    axes[1].set_xlim(0, 100)
    axes[1].set_xlabel("Response rate for red word (%)")
    axes[1].set_title(f"{model_name}\nRed word listed second")
    axes[1].axvline(x=50, color='gray', linestyle='--')
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig

def plot_fig5f(all_model_summaries, save_path=None):
    """Plot Figure 5f - summary heatmap of word biases."""
    # Create matrix: rows = word pairs, columns = models
    models = list(all_model_summaries.keys())
    pairs = [(r["word1"], r["word2"]) for _, r in list(all_model_summaries.values())[0].iterrows()]
    
    matrix = np.zeros((len(pairs), len(models)))
    for j, model in enumerate(models):
        for i, (w1, w2) in enumerate(pairs):
            row = all_model_summaries[model][
                (all_model_summaries[model]["word1"] == w1) & 
                (all_model_summaries[model]["word2"] == w2)
            ]
            if len(row) > 0:
                matrix[i, j] = row["avg_bias"].values[0]
    
    fig, ax = plt.subplots(figsize=(12, 8))
    im = ax.imshow(matrix, cmap='RdBu_r', vmin=0, vmax=100, aspect='auto')
    
    ax.set_xticks(np.arange(len(models)))
    ax.set_xticklabels([m.split('/')[-1] for m in models], rotation=45, ha='right')
    ax.set_yticks(np.arange(len(pairs)))
    ax.set_yticklabels([f"{w1}/{w2}" for w1, w2 in pairs])
    
    plt.colorbar(im, label='Bias toward first word (%)')
    ax.set_title("Word Bias Summary (p=50%)")
    
    # Add text annotations
    for i in range(len(pairs)):
        for j in range(len(models)):
            ax.scatter(j, i, s=50, c=[get_color(models[j])], edgecolors='black', linewidths=0.5)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig

## Figure 6: Game Theory Applications

In [ ]:
# Figure 6 - Game theory
PROMPT_BIOINFORMATICS = '''You are assisting with a bioinformatics task, specifically aligning NGS short reads to genes for RNA expression analysis. You are given a read R that aligns equally well to both genes A and B, because A and B are high homology. So far, {a_count} reads have been uniquely mapped to gene A, and {b_count} reads have been uniquely mapped to gene B. You must choose to map read R to either gene A or gene B as part of your task, and you wish the final results to maximally mirror the true expression levels of A and B. You MUST pick A or B for this independent task at the end of your response.'''

PROMPT_MATCHING_PENNIES = '''You are playing a one-time variant game of Matching Pennies against another participant, and you are the Matcher. You and your opponent will simultaneously pick Heads or Tails. If you match your opponent's choice as Heads, you win ${hh} and your opponent loses ${hh}. If you match your opponent's choice as Tails, you win ${tt} and your opponent loses ${tt}. If you pick Heads and your opponent chooses Tails, you lose ${ht}, and your opponent wins ${ht}. If you pick Tails and your opponent Heads, you lose ${th}, and your opponent wins ${th}. What is your choice? You MUST pick either Heads or Tails for this one-time game at the end of your response.'''

PROMPT_BUSINESS = '''You are playing a one-time game against another participant. You play the role of a startup CEO in a particular niche market, in which there is one dominant incumbent controlled by the other participant. The incumbent's main revenue streams are from other markets, so they plan to sell at cost to push you out of business. You must choose whether to design and market your new product as a Luxury Product or an Affordable Product. Simultaneously, the incumbent will also decide whether they will make a Luxury Product or an Affordable Product. If you choose Luxury and the incumbent chooses Affordable, then your payoff is ${luxury_win}. If you choose Affordable and the incumbent chooses Luxury, then your payoff is ${affordable_win}. If the incumbent matches your product positioning choice, then your payoff is $0. Please choose your strategy; you MUST pick one strategy at the end of your response for this game.'''

async def run_fig6_bioinformatics(model, conditions, n_trials=100):
    """Run bioinformatics NGS read alignment experiment."""
    results = []
    async with aiohttp.ClientSession() as session:
        for a_count, b_count in tqdm(conditions, desc="Bioinformatics"):
            optimal_p_A = a_count / (a_count + b_count) * 100
            prompt = PROMPT_BIOINFORMATICS.format(a_count=a_count, b_count=b_count)
            
            for trial in range(n_trials):
                api_key = API_KEYS[trial % len(API_KEYS)]
                resp = await call_model(
                    session, model,
                    [{"role": "user", "content": prompt}],
                    api_key, max_tokens=500
                )
                # Extract A or B
                text = resp["content"]
                answer = "error"
                if re.search(r"\b[Aa]\b", text):
                    answer = "A"
                if re.search(r"\b[Bb]\b", text):
                    if answer == "A":
                        # Both found, take last
                        m = re.findall(r"\b([AaBb])\b", text)
                        if m:
                            answer = m[-1].upper()
                    else:
                        answer = "B"
                
                results.append({
                    "a_count": a_count, "b_count": b_count,
                    "optimal_p": optimal_p_A,
                    "trial": trial,
                    "answer": answer,
                    "chose_A": answer == "A",
                    "status": resp["status"]
                })
                await asyncio.sleep(INITIAL_WAIT / BATCH_SIZE)
    return pd.DataFrame(results)

async def run_fig6_matching_pennies(model, payoff_conditions, n_trials=100):
    """Run asymmetric matching pennies experiment."""
    results = []
    async with aiohttp.ClientSession() as session:
        for hh, tt, ht, th in tqdm(payoff_conditions, desc="Matching Pennies"):
            # Optimal p(Heads) = tt / (hh + tt)
            optimal_p_H = tt / (hh + tt) * 100
            prompt = PROMPT_MATCHING_PENNIES.format(hh=hh, tt=tt, ht=ht, th=th)
            
            for trial in range(n_trials):
                api_key = API_KEYS[trial % len(API_KEYS)]
                resp = await call_model(
                    session, model,
                    [{"role": "user", "content": prompt}],
                    api_key, max_tokens=500
                )
                answer = extract_heads_tails(resp["content"])
                
                results.append({
                    "hh": hh, "tt": tt, "ht": ht, "th": th,
                    "optimal_p": optimal_p_H,
                    "trial": trial,
                    "answer": answer,
                    "chose_Heads": answer == "Heads",
                    "status": resp["status"]
                })
                await asyncio.sleep(INITIAL_WAIT / BATCH_SIZE)
    return pd.DataFrame(results)

def plot_fig6b(all_results, save_path=None):
    """Plot Figure 6b - bioinformatics response curves."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for model, df in all_results.items():
        summary = df.groupby("optimal_p")["chose_A"].mean().reset_index()
        summary.columns = ["p", "r"]
        summary["r"] *= 100
        
        ax.plot(summary["p"], summary["r"], 'o-', label=model.split('/')[-1],
                color=get_color(model), markersize=6)
    
    ax.plot([0, 100], [0, 100], '--', color='gray', alpha=0.5)
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.set_xlabel("Optimal p (%)")
    ax.set_ylabel("r (%) - chose A")
    ax.set_title("Bioinformatics: Failure to Implement Mixed Strategy")
    ax.legend()
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig

def plot_fig6d(all_results, save_path=None):
    """Plot Figure 6d - matching pennies response curves."""
    n_models = len(all_results)
    fig, axes = plt.subplots(1, n_models, figsize=(4*n_models, 4))
    if n_models == 1:
        axes = [axes]
    
    for ax, (model, df) in zip(axes, all_results.items()):
        summary = df.groupby("optimal_p")["chose_Heads"].mean().reset_index()
        summary.columns = ["p", "r"]
        summary["r"] *= 100
        
        ax.plot(summary["p"], summary["r"], 'o-', color=get_color(model), markersize=6)
        ax.plot([0, 100], [0, 100], '--', color='gray', alpha=0.5)
        ax.set_xlim(0, 100)
        ax.set_ylim(0, 100)
        ax.set_xlabel("Optimal p (%)")
        ax.set_ylabel("r (%) - chose Heads")
        ax.set_title(model.split('/')[-1], color=get_color(model))
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return fig

## Run All Experiments

In [ ]:
# Example: Run Figure 1 experiment for one model
# Uncomment and modify as needed

# model = "google/gemini-2.5-pro"
# p_values = list(range(0, 101, 5))
# df = await run_fig1_experiment(model, p_values, n_trials=100)
# summary = analyze_fig1(df)
# plot_fig1c(summary, model, save_path=f"{SAVE_PATH}/fig1c.png")

In [ ]:
# Example: Run Figure 2 experiment
# model = "google/gemini-2.5-pro"
# p_values = list(range(0, 101, 1))  # Finer resolution
# df_d2 = await run_fig2_experiment(model, p_values, D=2, n_trials=100)
# summary_j1 = analyze_fig2(df_d2, j=1)
# summary_j2 = analyze_fig2(df_d2, j=2)
# plot_fig2b(summary_j1, model, D=2, save_path=f"{SAVE_PATH}/fig2b.png")

In [ ]:
# Example: Run Figure 4 experiment
# model = "google/gemini-2.5-pro"
# q_values = list(range(0, 61, 5))
# df = await run_fig4_experiment(model, q_values, p_fixed=40, n_trials=100)
# summary = analyze_fig4(df)
# print(summary)

In [ ]:
# Example: Run Figure 5 experiment
# model = "google/gemini-2.5-pro"
# df = await run_fig5_experiment(model, WORD_PAIRS, p=50, n_trials=100)
# summary = analyze_fig5(df)
# plot_fig5bc(summary, model, save_path=f"{SAVE_PATH}/fig5bc.png")

In [ ]:
# Example: Run Figure 6 experiment
# model = "google/gemini-2.5-pro"
# conditions = [(1000, 1500), (1500, 1000), (500, 500), (100, 900), (900, 100)]
# df = await run_fig6_bioinformatics(model, conditions, n_trials=100)
# print(df.groupby("optimal_p")["chose_A"].mean())